# <span style="color:#2595bc">Dynamic Programming</span>

In [14]:
import numpy as np
import math
import time


In [18]:

#Jacks Car rental 
availability_rent = 10
moving_cost = 2

mean_request_first = 3
mean_request_second = 4

mean_return_first = 3
mean_return_second = 2

max_cars = 20
max_car_move = 5

no_of_states = (max_cars+1)*(max_cars+1)
no_of_possible_actions = (2*max_car_move + 1) #11

actions = np.arange(-max_car_move,max_car_move+1,1)

discount = .9


# current policy
policy = np.zeros((max_cars + 1, max_cars + 1))

# current state value
state_value = np.zeros((max_cars + 1, max_cars + 1))


#create an array of dicts
next_states = []

#Calculation of poisson takes a lot of time

lookup_poisson = dict()
def poisson(n,lambd):
    global lookup_poisson
    if n-lambd>=6:
        return 0
    else:
        key = lambd*10 + n
        if key not in lookup_poisson.keys():
            lookup_poisson[key] = math.exp(-lambd) * pow(lambd, n) / math.factorial(n)
        return lookup_poisson[key]

def expected_return(state, action, state_value):
    # initailize total return
    returns = 0.0

    # cost for moving cars
    returns -= moving_cost * abs(action)

    # go through all possible rental requests
    for rent_request_first in range(0, mean_request_first+6):
        for rent_request_second in range(0, mean_request_second+6):
            # moving cars
            n_cars_in_first = int(min(state[0] - action, max_cars))
            n_cars_in_second = int(min(state[1] + action, max_cars))

            # valid rental requests should be less than actual # of cars
            cars_rented_in_first = min(n_cars_in_first, rent_request_first)
            cars_rented_in_second = min(n_cars_in_second, rent_request_second)

            # get credits for renting
            reward = (cars_rented_in_first + cars_rented_in_second) * availability_rent
            n_cars_in_first -= cars_rented_in_first
            n_cars_in_second -= cars_rented_in_second

            # probability for current combination of rental requests
            prob = poisson(rent_request_first, mean_request_first) * \
                         poisson(rent_request_second, mean_request_second)

            
           
            placeholder_n_cars_in_first  = n_cars_in_first
            placeholder_n_cars_in_second = n_cars_in_second
            placeholder_prob = prob
            
            for returned_cars_in_first in range(0, mean_return_first+6):
                for returned_cars_in_second in range(0, mean_request_second+6):
                    
                    temp_n_cars_in_first = n_cars_in_first
                    temp_n_cars_in_second = n_cars_in_second
                    temp_prob = prob
                    temp_n_cars_in_first = min(temp_n_cars_in_first + returned_cars_in_first, max_cars)
                    temp_n_cars_in_second= min(temp_n_cars_in_second + returned_cars_in_second, max_cars)
                    temp_prob = poisson(returned_cars_in_first, mean_return_first) * \
                                poisson(returned_cars_in_second, mean_return_second) * prob
                    returns += temp_prob * (reward + discount * state_value[temp_n_cars_in_first, temp_n_cars_in_second])
    return returns



new_state_value = np.zeros((max_cars + 1, max_cars + 1))
improve_policy = False
#policyImprovementInd = 0
counter = 0

aa = time.time()
while True:
    
    counter += 1
    print ('counter ',counter)
    if improve_policy == True:
        alpha = time.time()
        # start policy improvement
        
        #print('Policy improvement', policyImprovementInd)
        print('Policy improvement')
        #policyImprovementInd += 1
        new_policy = np.zeros((max_cars + 1, max_cars + 1))
        for i in range(max_cars+1):
            for j in range(max_cars+1):
                action_returns = []
                # go through all actions and select the best one
                for action in actions:
                    if (action >= 0 and i >= action) or (action < 0 and j >= abs(action)):
                        action_returns.append(expected_return([i, j], action, state_value))
                    else:
                        action_returns.append(-float('inf'))
                best_action = np.argmax(action_returns)
                new_policy[i, j] = actions[best_action]

        # if policy is stable
        policyChanges = np.sum(new_policy != policy)
        #print('Policy for', policyChanges, 'states changed')
        beta = time.time()
        print("Policy improv took : ",beta-alpha)
        if policyChanges == 0:
            policy = new_policy
            break
            
        policy = new_policy
        improve_policy = False

    # start policy evaluation
    print('Policy Eval')
    for i in range(max_cars+1):
            for j in range(max_cars+1):
                new_state_value[i, j] = expected_return([i, j], policy[i, j], state_value)
    if np.sum(np.abs(new_state_value - state_value)) < 1e-4:
        state_value = new_state_value
        improve_policy = True
        continue
    state_value = new_state_value
    improve_policy = False
bb = time.time()

print('The whole process took : ',bb-aa)
print(policy)
print(state_value)

counter  1
Policy Eval
counter  2
Policy Eval
counter  3
Policy improvement
Policy improv took :  97.17668843269348
Policy Eval
counter  4
Policy improvement
Policy improv took :  104.92326855659485
Policy Eval
counter  5
Policy improvement


KeyboardInterrupt: 

array([-5, -4, -3, -2, -1,  0,  1,  2,  3,  4])